In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm_notebook

In [3]:
def fiscal_resample(df, fys, per):
    ret_p1 = df.ret_p1.cumprod().iloc[-1]
    retx_p1 = df.retx_p1.cumprod().iloc[-1]
    ret_p1_adj = df.ret_p1_adj.cumprod().iloc[-1]
    mv = df.mv.resample(fys).last().iloc[-1]
    res = pd.DataFrame({'ret_p1':ret_p1, 'retx_p1':retx_p1,
                        'ret_p1_adj':ret_p1_adj, 'mv':mv}, index=[per])
    return res

In [4]:
comp = pd.read_excel('demo.xlsx', sheet_name=0)
crsp = pd.read_excel('demo.xlsx', sheet_name=1, index_col=0)

In [20]:
# code_list = comp1.permno.unique().tolist()[:30]
# test_comp = comp1.set_index('permno').loc[code_list].reset_index()
# test_crsp = crsp.reset_index().set_index('permno').loc[code_list].reset_index().\
# set_index('date')
# test_comp.to_excel(writer, 'comp', index=False)
# test_crsp.to_excel(writer, 'crsp')
# writer = pd.ExcelWriter('demo.xlsx')
# writer.save()

In [5]:
tic = time.perf_counter()
g = comp.groupby(['permno', 'fyear'])
df_list = []
for name, group in tqdm_notebook(g):
    start = group.fysm.iloc[0].strftime('%Y-%m-%d')
    end = group.datadate.iloc[0].strftime('%Y-%m-%d')
    fystring = group.fystr.iloc[0]
#     print(name)
    sample = crsp[crsp.permno==name[0]][start:end]
    if len(sample) > 0:
        r = sample.resample(fystring)
        df_list.append(r.apply(fiscal_resample, fys=fystring, per=name[0]))

df = pd.concat(df_list, axis=0)
df.index.names = ['date', 'permno']

toc= time.perf_counter()
print(toc - tic)


6.8883288
